In [1]:
# Import required libraries

import tensorflow as tf
import pandas as pd

In [2]:
# Collecting data

url_tv_strategy = "http://161.97.165.206:57895/services/tv_internal_strategy"
data = pd.read_json(url_tv_strategy)

# data.columns = [
#     'time', 
#     'open', 
#     'high', 
#     'low', 
#     'close', 
#     'volume', 
#     'close_time', 
#     'quote_asset_volume', 
#     'number_of_trades', 
#     'taker_buy_base_asset_volume', 
#     'taker_buy_quote_asset_volume', 
#     'unused'
# ]

# data['date'] = pd.to_datetime(df['time'], unit='ms')
data.tail(5)

,time,Supertrend_$time,Supertrend_Plot,Supertrend_Up_Trend,Supertrend_Down_Trend,Supertrend_indicator,SMA_30_$time,SMA_30_MA,SMA_30_Smoothing_Line,SMA_30_indicator,...,candle_min,candle_volume,candle_indicator,round,round_position,round_position_value,round_close_at,round_lock_price,round_close_price,round_noroundid
996,1667563560,1667563560,344.7,1.000000e+100,347.255356,Supertrend,1667563560,347.813333,348.064667,SMA_30,...,344.1,229.07,candle,117674,Bull,1,2022-11-04 12:11:56,345.13,345.16554094,
997,1667563200,1667563200,345.7,1.000000e+100,348.292532,Supertrend,1667563200,348.063333,348.316000,SMA_30,...,345.5,190.05,candle,117673,Bear,0,2022-11-04 12:06:48,346.14617324,345.13,
998,1667563020,1667563020,345.6,1.000000e+100,348.292532,Supertrend,1667563020,348.180000,348.444667,SMA_30,...,345.4,127.21,candle,117672,Bear,0,2022-11-04 12:01:35,346.19692046,346.14617324,
999,1667562660,1667562660,345.9,1.000000e+100,348.296408,Supertrend,1667562660,348.453333,348.703333,SMA_30,...,344.9,1559.17,candle,117671,Bull,1,2022-11-04 11:56:26,345.83,346.19692046,
1000,1667562300,1667562300,346.7,1.000000e+100,348.775195,Supertrend,1667562300,348.706667,348.989333,SMA_30,...,346.4,250.75,candle,117670,Bear,0,2022-11-04 11:51:20,346.59502466,345.83,


In [3]:
# Remove unused columns
data.drop([
    'Supertrend_$time',
    'Supertrend_indicator',
    'SMA_30_$time',
    'SMA_30_indicator',
    'SMA_50_$time',
    'SMA_50_indicator',
    'SMA_100_$time',
    'SMA_100_indicator',
    'SMA_200_$time',
    'SMA_200_indicator',
    'EMA_30_$time',
    'EMA_30_indicator',
    'EMA_50_$time',
    'EMA_50_indicator',
    'EMA_100_$time',
    'EMA_100_indicator',
    'EMA_200_$time',
    'EMA_200_indicator',
    'RSI_$time',
    'RSI_indicator',
    'Stochastic_$time',
    'Stochastic_indicator',
    'Bollinger_Bands_$time',
    'Bollinger_Bands_indicator',
    'ATR_$time',
    'ATR_indicator',
    'candle_time',
    'candle_indicator',
    'round_position',
    'round_close_at',
    'round_lock_price',
    'round_close_price'], axis=1, inplace=True)

data.tail(5)

,time,Supertrend_Plot,Supertrend_Up_Trend,Supertrend_Down_Trend,SMA_30_MA,SMA_30_Smoothing_Line,SMA_50_MA,SMA_50_Smoothing_Line,SMA_100_MA,SMA_100_Smoothing_Line,...,Bollinger_Bands_Lower,ATR_ATR,candle_open,candle_close,candle_max,candle_min,candle_volume,round,round_position_value,round_noroundid
996,1667563560,344.7,1.000000e+100,347.255356,347.813333,348.064667,348.996,349.2520,348.402,348.4340,...,344.253403,0.901785,344.9,344.5,345.0,344.1,229.07,117674,1,
997,1667563200,345.7,1.000000e+100,348.292532,348.063333,348.316000,349.260,349.4968,348.444,348.4476,...,344.964426,0.857759,345.5,345.9,346.2,345.5,190.05,117673,0,
998,1667563020,345.6,1.000000e+100,348.292532,348.180000,348.444667,349.366,349.6228,348.444,348.4432,...,345.178133,0.864177,345.8,345.4,346.0,345.4,127.21,117672,0,
999,1667562660,345.9,1.000000e+100,348.296408,348.453333,348.703333,349.620,349.8632,348.457,348.4164,...,345.788825,0.903922,345.5,346.3,346.3,344.9,1559.17,117671,1,
1000,1667562300,346.7,1.000000e+100,348.775195,348.706667,348.989333,349.890,350.0668,348.422,348.3722,...,346.498135,0.687559,346.5,346.9,347.2,346.4,250.75,117670,0,
